# EDA 1st round

這是拿到初步整理資料後進行的前期探索，因為資料來源多種，有VD也有ETag，兩者的精度、粒度不同  
在交通領域上時間序列會跟路段型態有相當大的關係，這邊為了先簡化題目的範圍，會聚焦在國道五號的資料上  
EDA的過程是會來來回回的，敲定範圍後可能需要在對資料進行額外的加工後才能更進一步探索  
本notebook先做第一次的EDA，看的資料會直接以原始檔做為參考  

後續會以Etag的資料為主，因此VD的簡易帶過，讓大家能清楚結構跟差異

# Lib

In [1]:
import pandas as pd 
import sqlite3

部分的資料已經先儲存在DB中了，我們可以透過以下方式進行簡單的query  

In [3]:
db_path = "../data/hwdb.db"

# VD

## links and geo info

這邊的資料是前面儲存在VD Static的內容，主要記錄的是道路幾何型態資訊  
正常狀況下每天會是一樣的，除非該VD故障或者是被移除，通常這樣的資訊在較為長期的觀測中才會被放大  
如果我們要做的是預測路段的話，只要掌握link and node資訊即可
不太需要每天去鎖定VD的data  
VD的資料我們這邊做個簡易的探勘，帶大家看看在國道五號上相關的資料複雜程度  

In [2]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM VD_STATIC
    WHERE RoadName == "國道5號"
    ''',
    con)
con.close()

In [5]:
df.head(3).T

,0,1,2
UpdateTime,2023-11-01 00:00:00+08:00,2023-11-01 00:00:00+08:00,2023-11-01 00:00:00+08:00
UpdateInterval,86400,86400,86400
AuthorityCode,NFB,NFB,NFB
VDID,VD-N5-S-0.191-M-LOOP,VD-N5-S-12.945-M-LOOP,VD-N5-S-27.748-M-LOOP
SubAuthorityCode,NFB-PL,NFB-PL,NFB-PL
BiDirectional,0,0,0
LinkID,0000500000000A,0000500001200F,0000500002700G
Bearing,E,SE,S
RoadDirection,S,S,S
Lane,2,2,2


In [5]:
print(f'VDID 在國道五號上有 {df.VDID.nunique()} 個')
display(df.groupby(['RoadDirection', 'Lane', 'LocationType']).agg({'VDID': 'count'}))

VDID 在國道五號上有 140 個


VDID
RoadDirection Lane LocationType      
N             1    5               16
              2    5               52
              3    5                8
S             1    5               13
              2    5               46
              3    5                5

根據指定維度可以計算VD的數量，南北向數量不太一樣，各車道上的分布也不同  
從這邊可以先簡易的概括如果採用VD我們要研究的位置就相當複雜了，後面還涉及車種速率等指標的計算  
單一個方向就有76~64個偵測器，往下需要再通過link到link進行劃分，對於追求複雜的路網呈現  
或是要求更高的分析專案而言，可以此作為入門方向，只是後續要處理的資料工程難度更高  
本專案就不採納VD的資料作為研究方向，僅提供一個方向參考  

## traffic vol and status

這邊看的就會是VD Dynamic中存放的內容了  
基本上會有特定VD的資訊，放置的車道、設備狀態、偵測速率、佔有率等等  
因為資料量相當大，我們根據前一節的其中一個VDID切入大概看一下資料就好  

In [6]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM VD_DYNAMIC
    WHERE VDID == "VD-N5-S-0.191-M-LOOP"
    ''',
    con)
con.close()

In [8]:
display(df.shape)
df.head(7)

(8730, 14)

,UpdateTime,UpdateInterval,AuthorityCode,VDID,LinkID,LaneID,LaneType,Speed,Occupancy,VehicleType,Volume,Speed2,Status,DataCollectTime
0,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,-99,-99,S,-99,-99,1,2023-10-31 23:58:00+08:00
1,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,-99,-99,L,-99,-99,1,2023-10-31 23:58:00+08:00
2,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,-99,-99,T,-99,-99,1,2023-10-31 23:58:00+08:00
3,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,1,2,-99,-99,S,-99,-99,1,2023-10-31 23:58:00+08:00
4,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,1,2,-99,-99,L,-99,-99,1,2023-10-31 23:58:00+08:00
5,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,1,2,-99,-99,T,-99,-99,1,2023-10-31 23:58:00+08:00
6,2023-11-01 00:01:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,0,0,S,0,0,0,2023-10-31 23:59:00+08:00


目前只有塞一天的資料在裡面但不加條件的話數量是很驚人的，透過抽取特定VDID我們可以檢視一下  
其中就出現了好幾個資料異常  
這種異常說明了，如果要好好利用VD資料我們還需要針對這種異常進行缺補  
否則正常下我們可以直接就車種去計算加權平均的車速跟流量了  

In [26]:
df.VehicleType.value_counts()

VehicleType
S    2910
L    2910
T    2910
Name: count, dtype: int64

VD資料部分是依據車種拆開處理的，以我們展平數據的方式來看，相對更源頭的數據資料量會變成三倍

# ETag

接下來看核心的ETag資料

## Road and sensor position

In [8]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_STATIC
    ''',
    con)
con.close()

In [9]:
df.head()

,UpdateTime,UpdateInterval,AuthorityCode,LinkVersion,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile
0,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,03F2899N,0000300129000Q,4,120.48633,23.511683,000030,國道3號,0,N,竹崎(縣道159線),竹崎(縣道166線),289K+900
1,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0000S,0000501001000A,4,121.62302,25.035183,000050,國道5號,0,S,南港系統,石碇,0K+000
2,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,03F2306N,0000300123000M,4,120.70283,23.90678,000030,國道3號,0,N,南投服務區,南投,230K+600
3,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,01F3227N,0000100132300D,4,120.25,23.014172,000010,國道1號,0,N,大灣,永康,322K+700
4,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,03F1991S,0000300019900N,4,120.57766,24.102808,000030,國道3號,0,S,彰化系統,快官,199K+100


In [10]:
df['ETagGantryID'].nunique()

339

In [7]:
df.groupby(['RoadName', 'RoadDirection']).agg({'ETagGantryID': 'count'})

ETagGantryID
RoadName     RoadDirection              
國道1號         N                        75
             S                        75
國道1號五股楊梅高架道路 N                         4
             S                         4
國道1號汐止五股高架道路 N                         4
             S                         3
國道3甲         N                         2
             S                         2
國道3號         N                        78
             S                        78
國道5號         N                         7
             S                         7

本次聚焦在國道5號，因為從路線的幾何型態來看是相對簡單，不需要處理太多分支跟上下游分岔的狀況

與VD的幾何型態資訊一樣，在時間較長的範圍下，sensor的位置可會改變  
道路的長度概念上不會改變，但從資料面來說路線繪製的可能會有所調整    
LinkVersion如果把資料時間範圍拉長，就有機會觀測到版本變化(南北向的起始點有稍微不太一樣)  

In [24]:
# 這邊我們聚焦國五的北向
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    """
    SELECT * 
    FROM ETAG_STATIC
    WHERE RoadDirection == 'N'
    AND RoadID == '000050'
    """,
    con
)
con.close()

In [25]:
df

,UpdateTime,UpdateInterval,AuthorityCode,LinkVersion,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile
0,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0001N,0000500100000A,4,121.62472,25.035036,000050,國道5號,0,N,石碇,南港系統,0K+100
1,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0309N,0000500103100G,4,121.78635,24.823656,000050,國道5號,0,N,宜蘭(四城、大福),頭城,30K+900
2,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0528N,0000500105300G,4,121.80696,24.632729,000050,國道5號,0,N,蘇澳,羅東,52K+800
3,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05FR143N,0000501105010G,4,121.782364,24.733961,000050,國道5號,0,N,宜蘭(壯圍),宜蘭(四城、大福),14K+300
4,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0055N,0000500100500F,4,121.652084,24.996504,000050,國道5號,0,N,坪林交控中心專用道,石碇,5K+500
5,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0287N,0000500102820G,4,121.789185,24.842714,000050,國道5號,0,N,頭城,坪林交控中心專用道,28K+700
6,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0438N,0000500104400G,4,121.789474,24.711027,000050,國道5號,0,N,羅東,宜蘭(壯圍),43K+800


理論上國五的etc門架由北至南順序應該會是座路在這些地方中間  
* 南港系統
* 石碇
* 坪林交控中心專用道
* 頭城
* 宜蘭(四城、大福) 北入南出
* 宜蘭(壯圍) 南入北出
* 羅東
* 蘇澳

所以由北至南的順序應該會是  
* 05F0001N
* 05F0055N
* 05F0287N
* 05F0309N
* 05FR143N
* 05F0438N
* 05F0528N

In [26]:
df['SerialMile'] = df['LocationMile'].apply(lambda x: 
                                            int(x.split('+')[0].replace('K',''))*1000 
                                            + int(x.split('+')[1]))

In [27]:
# 手動只要排好下面的順序，可以用list + dict產生兌換用的功能
loc_list = ['蘇澳', '羅東', '宜蘭(壯圍)', '宜蘭(四城、大福)', '頭城', '坪林交控中心專用道', '石碇', '南港系統']
order_list = [i for i in range(len(loc_list))]
nd_order_dict = dict(zip(loc_list, order_list))


# 加上排序用cols
df['loc_order']=df['Start'].apply(lambda x: nd_order_dict[x])
df.sort_values(by=['loc_order'], inplace=True)

serial mile不太能作為順序參考，因為從地理上來說順序與位置未必成正向關係  

In [28]:
df

,UpdateTime,UpdateInterval,AuthorityCode,LinkVersion,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile,SerialMile,loc_order
2,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0528N,0000500105300G,4,121.80696,24.632729,000050,國道5號,0,N,蘇澳,羅東,52K+800,52800,0
6,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0438N,0000500104400G,4,121.789474,24.711027,000050,國道5號,0,N,羅東,宜蘭(壯圍),43K+800,43800,1
3,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05FR143N,0000501105010G,4,121.782364,24.733961,000050,國道5號,0,N,宜蘭(壯圍),宜蘭(四城、大福),14K+300,14300,2
1,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0309N,0000500103100G,4,121.78635,24.823656,000050,國道5號,0,N,宜蘭(四城、大福),頭城,30K+900,30900,3
5,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0287N,0000500102820G,4,121.789185,24.842714,000050,國道5號,0,N,頭城,坪林交控中心專用道,28K+700,28700,4
4,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0055N,0000500100500F,4,121.652084,24.996504,000050,國道5號,0,N,坪林交控中心專用道,石碇,5K+500,5500,5
0,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0001N,0000500100000A,4,121.62472,25.035036,000050,國道5號,0,N,石碇,南港系統,0K+100,100,6


In [29]:
target_etag_gantry = df.query("LinkVersion == '23.05.1' and RoadName == '國道5號'").ETagGantryID.unique()

## ETag pair info

從前面先取得門架的資訊，方便後續進行限縮

In [30]:
target_etag_gantry

array(['05F0528N', '05F0438N', '05FR143N', '05F0309N', '05F0287N',
       '05F0055N', '05F0001N'], dtype=object)

In [31]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_PAIR
    WHERE StartETagGantryID in (
        "05F0528N", "05F0438N", "05FR143N", "05F0309N", 
        "05F0287N", "05F0055N", "05F0001N")
        OR EndETagGantryID in (
        "05F0528N", "05F0438N", "05FR143N", "05F0309N", 
        "05F0287N", "05F0055N", "05F0001N")
    ''',
    con)
con.close()

In [33]:
df

,UpdateTime,UpdateInterval,AuthorityCode,ETagPairID,StartETagGantryID,EndETagGantryID,Description,Distance,StartLinkID,EndLinkID,Geometry
0,2023-11-01T00:00:00+08:00,86400,NFB,05F0438N-05F0309N,05F0438N,05F0309N,羅東-宜蘭(四城、大福),12.8,0000500104300G,0000500103000G,"LINESTRING(121.78873 24.71428,121.78849 24.715..."
1,2023-11-01T00:00:00+08:00,86400,NFB,05F0528N-05F0438N,05F0528N,05F0438N,蘇澳-羅東,8.76,0000500105200G,0000500104300G,"LINESTRING(121.80576 24.63708,121.80574 24.637..."
2,2023-11-01T00:00:00+08:00,86400,NFB,05F0055N-05F0001N,05F0055N,05F0001N,坪林交控中心專用道-石碇,5.28,0000500100500F,0000500100000A,"LINESTRING(121.65206 24.99663,121.65205 24.996..."
3,2023-11-01T00:00:00+08:00,86400,NFB,05F0309N-05F0287N,05F0309N,05F0287N,宜蘭(四城、大福)-頭城,2.13,0000500103090G,0000500102820G,"LINESTRING(121.78757 24.82461,121.78762 24.824..."
4,2023-11-01T00:00:00+08:00,86400,NFB,05F0438N-05FR143N,05F0438N,05FR143N,羅東-宜蘭(壯圍),0,0000500104300G,0000500101400F,"LINESTRING(121.78873 24.71428,121.78849 24.715..."
5,2023-11-01T00:00:00+08:00,86400,NFB,05F0287N-05F0055N,05F0287N,05F0055N,頭城-坪林交控中心專用道,22.95,0000500102820G,0000500100500F,"LINESTRING(121.78947 24.84347,121.78954 24.843..."


上述的資訊中包含了靜態的起訖對距離作為參考

## ETag pair dynamic info

這邊來看一下動態資訊中的變化吧  
把南北兩個方向都拉進來參考  

In [39]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_PAIR_LIVE
    WHERE ETagPairID in ('05F0438N-05F0309N', '05F0528N-05F0438N', '05F0055N-05F0001N',
                         '05F0439S-05F0494S', '05F0287S-05F0309S', '05F0309N-05F0287N',
                         '05F0438N-05FR143N', '05F0287N-05F0055N', '05FR113S-05F0439S',
                         '05F0309S-05F0439S', '05F0000S-05F0055S', '05F0055S-05F0287S')
    ''',
    con)
con.close()

In [40]:
df.head(3).T

,0,1,2
UpdateTime,2023-11-01 00:00:00+08:00,2023-11-01 00:00:00+08:00,2023-11-01 00:00:00+08:00
UpdateInterval,300,300,300
AuthorityCode,NFB,NFB,NFB
ETagPairID,05F0528N-05F0438N,05F0528N-05F0438N,05F0528N-05F0438N
StartETagStatus,0,0,0
EndETagStatus,0,0,0
VehicleType,31,32,41
TravelTime,374,332,0
StandardDeviation,0,0,0
SpaceMeanSpeed,86,95,0


請注意，上述的是原始資料，每五分鐘為單位計算  
包含了車種、旅行時間、空間平均速率、車輛數等統計  
後續我們可以使用其他一樣是ETag組成的資料進行分析  
這邊指大概帶過db擷取完畢的資料可以如何叫出跟解讀  

在進入本專案核心的EDA之前，因為硬體需求只能先將我需要觀察的資料拉出來另存成csv  
你想要用parquet也可以  

# Limit the scope of target data

以下大概註記一下DB這段我做了哪些事情  
儲存了多少跟擷取了多少資料出來  
好真正開始第二輪的EDA  

鎖定國道五號、往北方向的資料為主  
注意，我放在db中的資料量在預設是相當大的，因此實際拉出來應該會跟前面看到的不太一樣  

In [ ]:
# Etag location
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_STATIC
    WHERE RoadDirection == 'N'
    AND RoadID == '000050'
    ''',
    con)
con.close()

Etag_5n_loc = df.drop(columns=['UpdateInterval', 'AuthorityCode', 'RoadClass', 'LocationType']) \
                .drop_duplicates() \
                .copy()

Etag_5n_loc.to_csv('../data/cleaned/Etag_5n_loc.csv', index=False)

原始的M04A相當大，為了讓sqlite能負荷是以每個月作為一張資料表處理  
以下提供撈取的方法  

In [24]:
# M04A
table_time = ['202301', '202302' , '202303', '202304', '202305',
              '202306', '202307', '202308', '202309', '202310',
              '202311', '202312', '202401', '202402', '202403']

hw5_m04a = pd.DataFrame()

for year_month in table_time:
    db_table = 'ETAG_M04A_' + year_month
    con = sqlite3.connect("../data/hwdb.db")
    df = pd.read_sql_query(
        f'''
        SELECT * 
        FROM {db_table}
        WHERE GantryFrom in ('05F0438N', '05F0309N', '05F0528N', '05F0055N', '05F0001N',
        '05F0287N', '05FR143N', '03F0150N', '03F0201S')
        ''', con)
    con.close()
    hw5_m04a = pd.concat([hw5_m04a, df], axis=0)
    print(f'finish loading table {db_table}')

In [ ]:
hw5_m04a.to_csv('../data/cleaned/hw5_m04a.csv', index=False)

# Related data

這邊補充一些後續會用到的資料，部分有經過整理  

## section

In [47]:
section = pd.read_csv('../data/cleaned/section_info.csv')

In [49]:
section.SectionName.nunique()

608

In [51]:
section.groupby(['RoadName']).agg({'SectionName': 'nunique'})

,SectionName
RoadName,
南港連絡道,2
台2己,6
國2甲,2
國3甲,6
國道10號,14
國道1號,178
國道2號,12
國道3號,166
國道4號,14


In [52]:
section.head()

,SectionID,SubAuthorityCode,SectionName,RoadID,RoadName,RoadClass,RoadDirection,SectionLength,SpeedLimit,RoadSection_Start,RoadSection_End,SectionMile_StartKM,SectionMile_EndKM
0,1,NFB-NR,國道1號(基隆端到基隆交流道),000010,國道1號,0,S,1.1,40,基隆端,基隆交流道,0K+000,1K+100
1,3,NFB-NR,國道1號(基隆交流道到八堵交流道),000010,國道1號,0,S,1.5,54,基隆交流道,八堵交流道,1K+100,2K+600
2,5,NFB-NR,國道1號(八堵交流道到大華系統交流道),000010,國道1號,0,S,2.4,86,八堵交流道,大華系統交流道,2K+600,5K+000
3,419,NFB-NR,國道1號(大華系統交流道到五堵交流道),000010,國道1號,0,S,1.8,65,大華系統交流道,五堵交流道,5K+000,6K+800
4,7,NFB-NR,國道1號(五堵交流道到汐止交流道),000010,國道1號,0,S,3.7,133,五堵交流道,汐止交流道,6K+800,10K+500


## congestion_table

原始的國道壅塞道路時段表是一份pdf，裡面完全是人工整理好的表格，為了要能夠讓我們使用  
我手動製作了以下的csv file，好方便後續整合進訓練資料中，僅有國道五號方向保留  
該資料本身包含了國道方向、路段的起迄點、起始年月、壅塞統計上發生的時間段  

In [4]:
congestion_df = pd.read_csv('../data/cleaned/congestion_table.csv')

In [5]:
congestion_df

,road,direction,LinkStart,LinkEnd,StartYearMonth,EndYearMonth,dayofweek,CongestStart,CongestEnd
0,5,S,南港系統,頭城,202308,202403,weekday,900,1300
1,5,S,南港系統,頭城,202308,202403,Saturday,700,1600
2,5,N,宜蘭,坪林,202308,202403,weekday,1500,2000
3,5,N,宜蘭,坪林,202308,202403,Saturday,1600,2200
4,5,N,宜蘭,坪林,202308,202403,Sunday,1000,2400
5,5,N,羅東,宜蘭,202308,202403,Sunday,1300,1800
6,5,S,南港系統,頭城,202307,202307,weekday,900,1300
7,5,S,南港系統,頭城,202307,202307,Saturday,700,1600
8,5,N,宜蘭,坪林,202307,202307,weekday,1500,2000
9,5,N,宜蘭,坪林,202307,202307,Saturday,1600,2200


## calendar_event

節日活動的資訊也是個人整理出來的，涵蓋範圍從2022~2024的主要活動，在資訊上也標註了節日的涵蓋時間段範圍、是否為連續假日、假若為連續假日的話影響天數又有多長  
最長的就是農曆新年，為期 7 ~ 9天的連續假期  

In [6]:
calendar_event = pd.read_csv('../data/cleaned/calendar_event.csv')

In [7]:
calendar_event

,event_name,start_date,end_date,continuous,event_length
0,跨年元旦,2022-12-31,2023-01-02,T,3
1,農曆新年,2023-01-21,2023-01-29,T,9
2,元宵節,2023-02-05,2023-02-05,F,1
3,西洋情人節,2023-02-14,2023-02-14,F,1
4,二二八紀念日,2023-02-25,2023-02-28,T,4
5,婦女節,2023-03-08,2023-03-08,F,1
6,白色情人節,2023-03-14,2023-03-14,F,1
7,愚人節,2023-04-01,2023-04-01,F,1
8,兒童與清明節,2023-04-01,2023-04-05,T,5
9,復活節,2023-04-09,2023-04-09,F,1


## road_build_event

road build則是所謂的道路修繕資訊，這部分資料沒有做過多的清洗，可以看到在表頭的位置還區分為中英文對照(excel的關係)  
2023一整年下來的事件總數多達17萬  

In [14]:
road_build_event = pd.read_excel("../data/cleaned/202301_10_road_build_event.xlsx")

In [17]:
road_build_event

,incStepIncidentId,incStepNum,incStepTime,incStepEndTime,incStepFreewayId,incStepDirection,incStepStartMileage,incStepEndMileage,incStepBlockagePattern
0,事件編號,事件階段,階段開始時間,階段結束時間,路線,方向,起始里程,終止里程,占用車道
1,20230101074552939200021,1,2023-01-01 07:45:00,2023-01-01 16:06:00,10050,1,54300,54300,00000000010000000000
2,20230101082125894300045,1,2023-01-01 08:21:00,2023-01-01 08:25:00,10040,3,108,108,00000000010000000000
3,20230101084859011100021,1,2023-01-01 08:48:00,2023-01-01 16:41:00,10030,1,35900,35900,00000000010000000000
4,20230101084911770100021,1,2023-01-01 08:49:00,2023-01-01 16:41:00,10030,2,25650,25400,00000000010000000000
...,...,...,...,...,...,...,...,...,...
172217,20231031234150861100021,1,2023-10-31 23:41:00,2023-11-01 01:24:00,10030,2,27300,24700,01100000000000000000
172218,20231031234220062400021,1,2023-10-31 23:42:00,2023-11-01 00:24:00,10010,1,355000,355300,11000000000000000000
172219,20231031234402822100021,1,2023-10-31 23:44:00,2023-11-01 03:02:00,10030,1,61,478,10010000010000000000
172220,20231031234443502100021,1,2023-10-31 23:44:00,2023-11-01 02:06:00,10010,1,45000,46000,11100000000000000000


In [18]:
road_build_event.drop(index=0, inplace=True)

In [21]:
# 路段代碼意義
# 10010	國1
# 10018	高港高架
# 10019	國1高架
# 10020	國2
# 10030	國3
# 10031	國3甲
# 10040	國4
# 10050	國5
# 10060	國6
# 10080	國8
# 10100	國10

# 方向	
# 1	南
# 2	北
# 3	東
# 4	西

我們可以發現在路段施工部分，以國3(10030)位居榜首，因為國道在方向上的設計以奇數來說就是南北向  
偶數作為東西向的代表，所以不會有所謂一種國道編號會有四種方向同時存在   
而後續我們要鑽研的國道五號一整年來說道路修繕的統計也才3000多筆左右  

In [19]:
road_build_event.groupby(['incStepFreewayId', 'incStepDirection']).agg({'incStepIncidentId': 'nunique'})

incStepIncidentId
incStepFreewayId incStepDirection                   
10010            1                             20697
                 2                             19501
10018            3                                28
                 4                                17
10019            1                              1075
                 2                               997
10020            3                              1022
                 4                              1091
10030            1                             21929
                 2                             21835
10031            3                              1164
                 4                              1340
10038            1                              1086
                 2                              1190
10039            1                                29
                 2                                28
10040            3                              1431
                 4                              1595
10050            1                              3872
                 2                              3894
10060            3                              1467
                 4                              1424
10080            3                               910
                 4                               936
10100            3                              1562
                 4                              1418

## traffic_accident_data

這部分的資料則是交通事故通報的資料了，以格式上來說更為髒亂  
後續我另外撰寫了一個class去封裝其清洗的邏輯，這邊我們先單純的看統計即可  

In [10]:
traffic_accident_data = pd.read_excel("../data/cleaned/202301_10_traffic_accident_data.xlsx")

In [12]:
traffic_accident_data

,年,月,日,時,分,國道名稱,方向,里程,事件發生,交控中心接獲通報,...,車輛4,車輛5,車輛6,車輛7,車輛8,車輛9,車輛10,車輛11,車輛12,分局
0,2023,2,9,19,21,國道3號,南,54.0,19:21:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2023,2,10,3,36,國道1號,南,6.0,03:36:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2023,2,10,5,33,國道3號,南,76.0,05:33:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2023,2,10,7,35,國道3號,北,98.0,07:35:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2023,2,10,7,52,國道1號,北,23.0,07:52:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26734,2023,8,8,9,35,國道1號,南,23.0,09:35:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
26735,2023,8,8,9,44,國道3號,南,34.0,09:44:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
26736,2023,8,8,9,43,國道1號,北,6.0,09:43:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
26737,2023,8,8,9,42,國道1號,南,39.4,09:42:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [35]:
# 各國道上的事故次數統計，國道5號的總量其實不高
traffic_accident_data['國道名稱'].value_counts()

國道名稱
國道1號      16757
國道3號       7459
國道2號       1072
國道10號       537
國道5號        327
國道4號        149
國道6號        142
國道8號        122
國道3甲         76
國3甲          71
港西聯外道路       12
南港連絡道        10
國2甲           2
Name: count, dtype: int64

In [34]:
# 全國的事故統計，A3代表純車損，A2代表有人員受傷，A1代表死亡車禍
traffic_accident_data['事故類型'].value_counts()

事故類型
A3    22395
A2     1600
A1       59
Name: count, dtype: int64

In [41]:
traffic_accident_data.groupby(['事故類型']).agg({'處理分鐘': 'mean', '事件發生': 'count'})

,處理分鐘,事件發生
事故類型,,
A1,100.084746,59
A2,48.969375,1600
A3,14.591650,22395


越是嚴重的車禍，所要處理的時間越長

In [36]:
# 我們可以聚合一下幾個重要的維度來看一下
ta_df = traffic_accident_data.groupby(['國道名稱', '方向', '事故類型']).agg({'處理分鐘': 'mean', '事件發生': 'count'}).reset_index()

In [37]:
ta_df.sort_values(by='事件發生', ascending=False)

,國道名稱,方向,事故類型,處理分鐘,事件發生
15,國道1號,北,A3,13.474038,6394
21,國道1號,南,A3,13.989794,4801
37,國道3號,北,A3,14.420620,3288
43,國道3號,南,A3,14.749004,2008
24,國道1號,南向,A3,14.636891,1724
...,...,...,...,...,...
1,南港連絡道,北,A3,88.000000,1
71,國道8號,雙向,A2,22.000000,1
49,國道4號,東,A2,68.000000,1
2,南港連絡道,南,A2,29.000000,1


In [40]:
# 如果僅看國道五號的話，會是如何?
ta_df[ta_df['國道名稱'] == "國道5號"]

,國道名稱,方向,事故類型,處理分鐘,事件發生
56,國道5號,北,A2,46.625000,16
57,國道5號,北,A3,14.668712,163
58,國道5號,南,A2,45.578947,19
59,國道5號,南,A3,21.573643,129
